In [1]:
import pandas as pd
import pyodbc
import os
import useful_functions as use

In [2]:
username = 'vigrose'
password = 'Ravenclaw~10946'
s = "DSN=PRDDM; UID={}; PWD={}".format(username, password)
AMADM = pyodbc.connect(s)

In [5]:
order_query= \
    f""" 
    SELECT DISTINCT 
    D.FULL_DT,
    H.MED_EDU_NBR AS ME,
    H.PARTY_ID,
    O.ORDER_NBR,
    O.ORDER_PRODUCT_ID, 
    O.ORDER_PHYSICIAN_HIST_KEY,
    O.CUSTOMER_KEY
    FROM 
    AMADM.DIM_DATE D, AMADM.FACT_EPROFILE_ORDERS O, AMADM.DIM_PHYSICIAN_HIST H
    WHERE  
    D.DATE_KEY = O.ORDER_DT_KEY
    AND 
    H.PHYSICIAN_HIST_KEY = O.ORDER_PHYSICIAN_HIST_KEY
    AND
    D.YR in (2019,2020,2021)
    """
#define customer query
customer_query = \
    """
    SELECT DISTINCT
    C.CUSTOMER_KEY, 
    C.CUSTOMER_NBR,
    C.CUSTOMER_ISELL_LOGIN,
    C.CUSTOMER_NAME,
    C.CUSTOMER_TYPE_DESC,
    C.CUSTOMER_TYPE,
    C.CUSTOMER_CATEGORY_DESC
    FROM 
    AMADM.dim_customer c  
    """

In [6]:
# orders = pd.read_sql(con=AMADM, sql=order_query)
customers = pd.read_sql(con=AMADM, sql=customer_query)
customers.CUSTOMER_KEY = customers.CUSTOMER_KEY.astype(str)
customers = customers.fillna('None')
orders = orders[orders.ORDER_PRODUCT_ID.isin([4915513,4915514])]

In [8]:
x = "DSN=prdsso; UID={}; PWD={}".format(username, password)
SSO = pyodbc.connect(x)

org_query = \
        '''
        SELECT *
        FROM
        SSO.ORGANIZATION
        '''
org_addresses = pd.read_sql(con=SSO, sql=org_query)
org_addresses['ADVANTAGE_ID'] = org_addresses.ADVANTAGE_ID.fillna('0')
org_addresses = org_addresses[org_addresses.ADVANTAGE_ID!='']
org_addresses['CUSTOMER_NBR']=[float(x) for x in org_addresses.ADVANTAGE_ID]
org_addresses = org_addresses.sort_values('STATUS').drop_duplicates('ADVANTAGE_ID', keep='last')

In [14]:
customers.CUSTOMER_NBR = customers.CUSTOMER_NBR.astype(float)

In [16]:
customers = pd.merge(org_addresses, customers, on='CUSTOMER_NBR')

In [18]:
RESULTS = pd.merge(customers, orders, on='CUSTOMER_KEY')

In [46]:
hosp = RESULTS[RESULTS.CUSTOMER_CATEGORY_DESC=='Hospital']

In [47]:
top = hosp.groupby('CUSTOMER_KEY').count().sort_values('CUSTOMER_NAME', ascending=False)[0:100]

In [52]:
customers[customers.CUSTOMER_KEY.isin(list(top.index))][25:50]

,ORG_ID,ORG_NAME,PARTY_ID,STATUS,ADVANTAGE_ID,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIPCODE,ORGANIZATION_TYPE,PHONE_NUMBER,CUSTOMER_NBR,CUSTOMER_KEY,CUSTOMER_ISELL_LOGIN,CUSTOMER_NAME,CUSTOMER_TYPE_DESC,CUSTOMER_TYPE,CUSTOMER_CATEGORY_DESC
8255,994,Select Medical,None,1,000002081225,500 Riverhills Business Park Ste 500,,Birmingham,AL,352425039,HP ...,2059955668,2081225.0,60051,01HP71782,Select Medical,Eprofile Webstore Customer,Eprofile,Hospital
8280,990,HCA - Nashville Credentialing Processing Center,None,1,000002081221,552 Metroplex Dr,,Nashville,TN,372113133,HP ...,8663761045,2081221.0,53216,47HP64899,Hca - Nashville Credentialing Processing Center,Eprofile Webstore Customer,Eprofile,Hospital
8412,1065,Hoag Hospital,None,1,000002081308,1 Hoag Dr,,Newport Beach,CA,926634162,HP ...,9497644624,2081308.0,57043,05HP73446,Hoag Hospital,Eprofile Webstore Customer,Eprofile,Hospital
8422,1062,Sutter Shared Services CVO - S3 CVO,None,1,000002081305,9100 Foothills Blvd,,Roseville,CA,957475888,HP ...,5102042430,2081305.0,60323,05HP50911,Sutter Shared Services Cvo - S3 Cvo,Eprofile Webstore Customer,Eprofile,Hospital
8512,256,West Virginia University Health System,None,1,000002080145,Medical Center Drive,Box 8133,Morgantown,WV,265068133,CVO ...,3045984156,2080145.0,132928,55HP07279,West Virginia University Health System,Eprofile Webstore Customer,Eprofile,Hospital
8528,251,University of Colorado Hospital,None,1,000002080139,Campus Box A-042,12401 E 17th Ave,Aurora,CO,800452548,HP ...,7208488587,2080139.0,53079,07HP06885,University Of Colorado Hospital,Eprofile Webstore Customer,Eprofile,Hospital
8620,201,Contra Costa Regional Medical Center,None,1,000002080089,2500 Alhambra Ave,,Martinez,CA,945533152,HP ...,9253705116,2080089.0,54588,05HP05928,Contra Costa Regional Medical Center,Eprofile Webstore Customer,Eprofile,Hospital
8661,314,Parkland Memorial Hospital,None,1,000002080204,5201 Harry Hines Blvd,,Dallas,TX,752357793,HP ...,2145908080,2080204.0,52277,48HP06306,Parkland Health & Hospital System,Eprofile Webstore Customer,Eprofile,Hospital
8662,314,Parkland Memorial Hospital,None,1,000002080204,5201 Harry Hines Blvd,,Dallas,TX,752357793,HP ...,2145908080,2080204.0,134720,48HP06306,Parkland Memorial Hospital,Eprofile Webstore Customer,Eprofile,Hospital
8712,343,St. Francis Health System,None,1,000002080235,1 Saint Francis Dr,,Greenville,SC,296013955,HP ...,8642551584,2080235.0,62443,45HP06544,Bon Secours St. Francis Health System,Eprofile Webstore Customer,Eprofile,Hospital


In [43]:
hospitals = pd.read_excel('../../Data/Rosters/Hospital_Scraping_Progress.xlsx')